# **Docker**

**Dockerfile** — текстовый файл, в котором описывается процесс сборки Docker-образа. В нем указываются инструкции для скачивания необходимых компонентов, копирования файлов, настройки среды и многие другие действия, необходимые для создания окружения, в котором будет работать ваше приложение.

**Docker image** — шаблон, на основе которого запускается контейнер Docker. Он содержит все необходимые файлы для запуска приложения в изолированном окружении. Docker images создаются на основе Dockerfile и могут быть сохранены и перенесены между разными средами.

**Docker registry** — это сервис, который хранит и управляет Docker images. Он позволяет пользователям загружать и скачивать образы, делиться ими с другими пользователями и управлять доступом к образам. Docker Hub, о котором я упоминал ранее, является одним из наиболее известных публичных Docker реестров.

### **Основные команды**

In [ ]:
docker ps #проверить запущенные контейнеры
docker ps -a #проверить запущенные контейнеры, включая остановленные
service docker status #проверка запущен ли Docker (LINUX)

docker run hello-world #создание контейнера
docker rm c6ae #удаление контейнера по первым символам его CONTAINER ID

docker images #просмотр images
docker rmi feb5 #удаление image по первым с имволам его IMAGE ID

docker pull ubuntu #установка образа (image) c Docker Hub

docker run ubuntu #создание и запуск контейнера на основе образа
docker run ubuntu sleep 5 #создание и запуск контейнера на пять секунд
docker run -d ubuntu sleep 10 #создание и запуск контейнера в фоне
docker run ubuntu:20.04 #создание и запуск контейнера конкретной версии с Docker Hub

docker start 644o #запуск контейнера 

docker pause 1ce #пауза контейнера
docker unpause 1ce #снятие контейнера с паузы

docker stop 403t #остановка контейнера
dokker kill 403t #убить контейнер

docker run --rm -d ubuntu sleep 900 #после остановки контейнер будет убит
docker run -d --rm nginx

docker inspect 150x #информация о контейнере
docker stats 150x #информация о нагрузке на память и CPU
docker run --rm --name My-Container ubuntu echo "Hello!" #дать имя контейнеру и передача аргумента

docker logs MyNginx #просмотр лога контейнера по его имени
docker logs -f MyNginx #обновляющийся лог

docker exec -it MyNginx /bin/bash #интерактивный режим, выполнение команд внутри контейнера: открытие bash
exit #выход из интерактивного режима

docker system prune -a --volumes #удаление всех ОСТАНОВЛЕННЫХ контейнеров всех образов
docker run -d --name web1 -p 80:80 nginx #пробрасывание порта 80 сервера на порт 80 докера

ip a #посмотреть айпи порта (LINUX)

### **Переменные**

In [ ]:
docker run --name mysql1 -e MYSQL_ROOT_PASSWORD=pw1234 -d mysql 
       #создание контейнера с именем и переменной окружения в фоновом режиме
docker exec -it mysql1 /bin/bash #интерактивный режим, где можем проверить наличие переменной
                 #командой env
mysql -p #подключение с вводом пароля

docker exec -it mysql1 mysql -uroot -p #можно сразу войти таким образом

### **Постоянные данные (Volumes)**

Монтирование в контексте Docker означает привязку файловой системы хост-системы к файловой системе внутри контейнера. Это позволяет обмениваться данными между хост-системой и контейнером, сохраняя данные после остановки или удаления контейнера.

In [ ]:
docker volume ls #проверить список volume'ов
docker run --name web01 -p 80:80 -d nginx #пробрасывание порта

docker exec -it web01 /bin/bash #внесем некоторые изменения
 cd /usr/share/nginx/html
 ls -la
 cat index.html
 cat <<EOT >>index.html #вводим html код
 cat index.html #просматриваем изменения. на порте html изменился
                #после удаления этого контейнера и создания одноименного
                #внесенные изменения пропадут
----------------------------------------------------------------------------
#Как фиксить?
----------------------------------------------------------------------------
#Host Volumes
cd /opt/
mkdir nginx
cd nginx
mkdir data

docker run --name web01 -p 80:80 -v /opt/nginx/data/:/usr/share/nginx/html -d nginx #монтируем новый каталог в каталог контейнера
#на сайте ничего не показывается, так как новая директория пуста

cd data
vi index.html #открываем редактор и копируем сюда содержимое корневого html
              #теперь сайт работает

docker stop web01
docker rm web1
docker run --name web01 -p 80:80 -v /opt/nginx/data/:/usr/share/nginx/html -d nginx
#после удаления и создания одноименного контейнера изменения сохранились
----------------------------------------------------------------------------
#Anonymous Volumes

cd /var/lib/docker/volumes

docker run --rm --name web01 -p 80:80 -v /usr/share/nginx/html -d nginx
ls #видим контейнер типа 234redwf4r84ri4ri5434r34r
cd 234redwf4r84ri4ri5434r34r/
ll
cd _data
vi index.html #вставляем текст, в итоге видим содержимое на сайте
----------------------------------------------------------------------------
#Named Volumes (САМОЕ РАСПРОСТРАНЕННОЕ)
docker run --rm --name web01 -p 80:80 -v web_data:/usr/share/nginx/html -d nginx
cd /var/lib/docker/volumes/web_data/_data
#при остановке докера data останется
----------------------------------------------------------------------------
docker volumes create infos #создание volume'а
docker volume ls #список volume'ов
docker volume rm infos #удаление volume'а

### **Сети**

In [ ]:
docker network ls #список сетей

docker network create myNet01 #создание сети типа bridge
docker network create --driver host myNethost #может быть единственная сеть типа host
docker network create -d null myNethost #может быть единственная сеть типа null
docker network create -d bridge --subnet 192.168.10.0/24 --gateway 192.168.10.1 myNet192 #создание сети по конкретному адресу

docker network inspect myNet01 #сведения о сети
docker inspect container1 #сведения о контейнере, узнать ip контейнера

docker network rm myNet01 #удаление сети

docker run --rm -it --name cont1 nicolaka/netshoot /bin/bash #запуск специального контейнера, имеющего инструменты для работы с сетью

docker run --rm -it --name cont1 --net myNet01 nicolaka/netshoot /bin/bash #запуск контейнера в сети myNet01
docker run --rm -it --name cont2 --net myNet01 nicolaka/netshoot /bin/bash #запуск второго контейнера в сети myNet01

#контейнеры находятся в одной сети, мы можем получить пинг одного контейнера через другой
#это можно сделать по ip или по имени
ping 172.20.0.2
ping cont1

docker network connect myNet01 cont2 #подключить контейнер к сети
docker network disconnect 4nfcji55902mcnfjft95dcnb5820rjf49501 cont2 #отключиться от сети по NetworkID, полученной с помощью docker inspect cont2

### **Создание собственных images**

#### **Пример 1: Ubuntu контейнер**

In [ ]:
mkdir docker
cd docker
touch dockerfile
vi dockerfile

**#---------------------содержимое файла---------------------#**
FROM ubuntu:22.04     #на чем будет базироваться образ

LABEL autor=Ilya    #описание

RUN apt-get update
RUN apt-get install nginx -y #-y - автоматическое согласие на установку

WORKDIR /var/www/html/ #рабочая директория

ENV OWNER=Ilya #создание переменных, их видно при работе через exec после ввода команды env
ENV TYPE=demo #к ним можно обращаться через $OWNER/$TYPE

COPY files/index.html . #копирование файлов в рабочую директорию

EXPOSE 80 #указание под каким портом работать с контейнером 
#ENTRYPOINT ["echo"]
CMD ["nginx", "-g", "daemon off;"] #запуск nginx с параметрами

#ENTRYPOINT - неизменяемая команда, а CMD - изменяемая
**#---------------------содержимое файла---------------------#**
cd ..
mkdir files
cd files
vi index.html
cd ..
****
docker build -t myimage:v01 #создание собственного образа
docker images #ищем идентификатор
docker tag 94uerv0452 mydocker:v01 #даём имя image'у
docker run -d --rm --name mydocker -p 80:80 mydocker:v01 #создание контейнера на основе созданного образа
docker image inspect mydocker:v01 #сведения об образе, тут же видны LABEL's

#### **Пример 2: Apache контейнер**

In [ ]:
mkdir apache
cp dockerfile apache/ #копируем в новую папку докерфайл из прошлого п
cd apache
vi dockerfile

**#---------------------содержимое файла---------------------#**
FROM ubuntu:22.04     #на чем будет базироваться образ

LABEL autor=Ilya    #описание

RUN apt-get update
RUN apt-get install apache2 -y #-y - автоматическое согласие на установку

EXPOSE 80 #указание под каким портом работать с контейнером 

CMD ["apache2ctlctl", "-D", "FOREGROUND"]

**#---------------------содержимое файла---------------------#**

docker build -t myapache:v01
docker run -d --rm --name mydocker -p 80:80 myapache:v01

#### **Пример 3: Python контейнер**

In [ ]:
mkdir python
cd python
touch sum.py
vi sum.py

**#---------------------содержимое файла---------------------#**
num1 = int(input())
num2 = int(input())

sum = num1 + num2

print('The sum of {num1} and {num2} is {sum}')
**#---------------------содержимое файла---------------------#**

touch dockerfile
vi dockerfile

**#---------------------содержимое файла---------------------#**
FROM python:3
WORKDIR /usr/src/app
COPY sum.py ./
CMD ["python", "./sum.py"]
**#---------------------содержимое файла---------------------#**

docker build -t mypython:v01
docker run -it --rm mypython:v01

### **Docker Compose**

Docker Compose используется для управления одним или несколькими контейнерами, содержит инструкции по запуску контейнеров, упрощает автоматизацию запуска контейнеров. Он описывается в YAML файле (docker-compose.yml).

#### **Пример 1:**

In [ ]:
ocker compose --version

cd /opt/
mkdir compose
cd compose
touch docker-compose.yml
vi docker-compose.yml

**#---------------------содержимое файла---------------------#**
version: '3.5'
services:
 web-server:
  image: nginx:stable
  container_name: mynginx
  volumes:
  - '/opt/web/html:/var/www/html' #выделенные директории создаются автоматически
  - '/opt/web/pics:/var/www/pictures'
  - 'nginx-config:/var/www/config' #именные волюмы описывают ниже
 environment:
  - 'NGINX_HOST=web.romnero.de'
  - 'NGINX_PORT=80'
 ports:
  - '80:80'
  - '433:433'
 restart: unless-stopped
volumes: #тут описываются именные волюмы
 nginx-config

networks:
 default:
  driver: bridge
  name: webnet
**#---------------------содержимое файла---------------------#**

docker compose up -d #запуск
docker compose logs -f #увидить логи
docker compose stop #остановка

#### **Пример 2:**

In [ ]:
**#---------------------содержимое файла---------------------#**
version: '3.6'
services:
 php-app:
  image: php:apache
  container_name: app
  ports:
   - '80:80'
  restart: unless-stopped
  repends_on:
   - app-db
   - app-redis
  networks:
   - internet
   - localnet
   
app-db:
 image: postgres
 container_name: app-postgres
 restart: unless-stopped
 environment:
  - 'POSTGRES_PASSWORD=mysecretpassword'
 networks: localnet
 
app-redis:
 image: redis
 container_name: app-redis
 restart: unless-stopped
 networks: 
  - localnet
 
 networks:
  internet:
   name: internet
   driver: bridge
  localnet:
   name: localnet
   driver: bridge 
**#---------------------содержимое файла---------------------#**
   

docker compose up -d #запуск